# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.  

**Описание данных:**  
children — количество детей в семье  
days_employed — общий трудовой стаж в днях  
dob_years — возраст клиента в годах  
education — уровень образования клиента  
education_id — идентификатор уровня образования  
family_status — семейное положение  
family_status_id — идентификатор семейного положения  
gender — пол клиента  
income_type — тип занятости  
debt — имел ли задолженность по возврату кредитов  
total_income — ежемесячный доход  
purpose — цель получения кредита  

## Предобработка данных

In [1]:
import pandas as pd

In [2]:
try:
    df = pd.read_csv('data.csv')
except:
    df = pd.read_csv('/datasets/data.csv')

In [3]:
df.info() #общая информация от датасете

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
df.head(15) #взглянув на первые строки можно заметить значительную часть проблем представленных в датасете

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В таблице присутствую пропуски, а также странные значения в графе трудового стажа.
Для начала рассмотрим пропущенные значения:

In [5]:
df.isna().sum() # проверяем в каких столбцах имеются пропуски

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Количество пропусков в столбцах days_employed и total_income совпадает.
Можно предположить, что базе данных просто не выставлены нулевые значения, поскольку 
у безработных текущий стаж будет равен нулю, следовательно и доход будет нулевым.
Также всегда есть вероятность банального пропуска значений.
Проверим строки с пустыми значениями на вид деятельности:

In [6]:
# посмотрим таблицу из строк только с пропущенными значениями
empty_data = df[df['days_employed'].isna()]
empty_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


При осмотре рядов с отсутствующими значениями, можно сделать вывод, что пропущены не нулевые значения. Стаж и доход отсутствуют не только у безработных и пенсионеров.

Поскольку мы не располагаем достаточно точными параметрами для заполнения этих столбцов при помощи сторонней информации(средние данные по результатам опросов/наблюдений),
заполним их значениями из самого датасета. 
Для столбца 'days_eployed' подойдет заполнение медианным значением, поскольку часть значений значительно отличается от большинства. Для данных столбца 'total_income' подойдут как среднее арфиметическое значение, так и медиана. Поскольку логичнее будет спользовать один метод для обоих столбцов, сделаем выбор в пользу медианы.

In [7]:
df['total_income'] = df['total_income'].fillna(df['total_income'].median())

Большинство значений в столбце 'days_employed' являются отрицательными, что скорее всего является следствием механической ошибки. Приведем значения столбца к модулю, перед вычислением медианы.

In [8]:
df['days_employed'] = df['days_employed'].abs() #модуль
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median()) #медиана

Перейдем к стобцу 'children'

In [9]:
df['children'].unique() # смотрим уникальные значения в стобце

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Отрицательное количество детей похоже на явную ошибку.
Хотя человек может иметь 20 детей, предположим, что это просто опечатка и реальное значение должно быть 2.

In [10]:
df['children'] = df['children'].replace(20, 2) #сначала исправляем ошибку с завушенным значением
df['children'] = df['children'].abs() #модулем избавляеся от отрицательных величин
df['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Перейдем к дубликатам

In [11]:
df.duplicated().sum() #изначальное количество явных дубликатов

54

In [12]:
df['education'].unique() #в этом столбце сразу видны проблемы с ригистром

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [13]:
df['education'] = df['education'].str.lower() #приведение всех строк к нижнему регистру не только добавит эстетики, но и поможет в выявлении дубликатов

In [14]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [15]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [16]:
df['family_status'] = df['family_status'].str.lower() #изменим регистр просто ради хорошего стиля

In [17]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [18]:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [19]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
df.duplicated().sum()

71

In [21]:
df = df.drop_duplicates().reset_index(drop=True) # в итоге избавляемся от всех явных дубликатов сразу
df.duplicated().sum()

0

In [22]:
df['total_income'] = df['total_income'].astype('int') #переведем столбец 'total_income' в формат цельных чисел для удобства
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Избавимся от отображения столбцов 'family_status' и 'education', оставив только их идентификаторы, а сами значения поместим в своеобразные словари.

In [23]:
family_dict = df[['family_status', 'family_status_id']] #создаем таблицу из двух столбцов 
family_dict.head(10)

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1
5,гражданский брак,1
6,женат / замужем,0
7,женат / замужем,0
8,гражданский брак,1
9,женат / замужем,0


In [24]:
family_dict = family_dict.drop_duplicates().reset_index(drop=True) #оставляем только уникальные значения

In [25]:
family_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [26]:
education_dict = df[['education', 'education_id']]
education_dict.head(10)

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1
5,высшее,0
6,высшее,0
7,среднее,1
8,высшее,0
9,среднее,1


In [27]:
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [28]:
#удаляем лишние столбцы из основной таблицы
del(df['education'])
del(df['family_status'])

In [29]:
df.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


Разобьем заемщиков на категории в соответствии с их доходами, в порядке убывания, где A - имеющие наибольший доход, а E - соответсвенно наименьший.

In [30]:
#простая формула для присвоения категорий
def category_check(total_income):
    if total_income <= 30000:
        return 'E'
    if 30000 < total_income <= 50000:
        return 'D'
    if 50000 < total_income <= 200000:
        return 'C'
    if 200000 < total_income <= 1000000:
        return 'B'
    return 'A'
df['total_income_category'] =  df['total_income'].apply(category_check)
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


Теперь сгруппируем цели кредита в четыре основных категории: 'операции с автомобилем','операции с недвижимостью', 'проведение свадьбы' и 'получение образования'

In [31]:
# поиск для недвижимости смотрится слегка костыльно, но с данным набором значений работает
def purpose_check(purpose):
    if "авто" in purpose:
        return 'операции с автомобилем'
    if "жи" in purpose:
        return 'операции с недвижимостью'
    if "свад" in purpose:
        return 'проведение свадьбы'
    return 'получение образования'
df['purpose_type'] = df['purpose'].apply(purpose_check)
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_type
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


## Анализ данных

Для начала построим сводную таблицу для просроченных кредитов в зависимости от семейного пложения заемщика.

In [32]:
family_pivot = df.pivot_table(index='family_status_id', columns='debt', values='gender', aggfunc='count') #строим сводную таблицу
family_pivot['share'] = family_pivot[1] /family_pivot[0] #добавляем столбец с долей просроченных кредитов
family_pivot['status'] = family_dict['family_status'] #достаем полноценные значения для большей наглядности
family_pivot

debt,0,1,share,status
family_status_id,,,,
0,11408,931,0.081609,женат / замужем
1,3763,388,0.103109,гражданский брак
2,896,63,0.070312,вдовец / вдова
3,1110,85,0.076577,в разводе
4,2536,274,0.108044,не женат / не замужем


Наиболее очевидный вывод: люди состоящие или состоявшие в официальном браке, с меньшей вероятностью просрочат кредит.

Проделаем схожую процедуру, теперь по количеству детей.

In [33]:
children_pivot = df.pivot_table(index='children', columns='debt', values='gender', aggfunc='count')
children_pivot = children_pivot.fillna(0) #все работает из пустым значение в пятой строке, но заменим её нулем для наглядности
children_pivot['share'] = children_pivot[1] / children_pivot[0]
children_pivot

debt,0,1,share
children,,,
0,13028.0,1063.0,0.081593
1,4410.0,445.0,0.100907
2,1926.0,202.0,0.104881
3,303.0,27.0,0.089109
4,37.0,4.0,0.108108
5,9.0,0.0,0.000000


Вывод: заемщики с детьми чаще просрочивают платежи, нулевой коэффициент должников у людей с пятью детьми - аномалия связанная с недостаточной выборкой.

Хотя заказчик об этом не просил, у нас есть возможность без лишних усилий проверить влияние цели кредита на возможную просрочку.

In [34]:
purpose_pivot = df.pivot_table(index='purpose_type', columns='debt', values='gender', aggfunc='count')
purpose_pivot['share'] = purpose_pivot[1] / purpose_pivot[0]
purpose_pivot

debt,0,1,share
purpose_type,,,
операции с автомобилем,3903,403,0.103254
операции с недвижимостью,10029,782,0.077974
получение образования,3643,370,0.101565
проведение свадьбы,2138,186,0.086997


Наиболее ответсвенно подходят к выплатам, люди взявшие кредит для операций с недвижимостью.
Автомобилисты чаще других просрачивают платежи, можно предположить, что это из-за непредвиденных расходов связанных с самим автомобилем(штрафы, ремонт)

Попробуем также оценить зависимость просрочки кредита от финансового благосостояния заемщика.

In [35]:
#этот вариант проверяю, только потому что это написано в инструкции.
income_pivot = df.pivot_table(index='total_income_category', columns='debt', values='gender', aggfunc='count')
income_pivot['share'] = income_pivot[1] / income_pivot[0]
income_pivot

debt,0,1,share
total_income_category,,,
A,23,2,0.086957
B,4685,356,0.075987
C,14656,1360,0.092795
D,329,21,0.063830
E,20,2,0.100000


Увы, данные из этой таблицы трудно назвать полезными. Разброс в количестве заемщиков слишком велик, к тому же этот параметр подвергался автозаполнения, что допольнительно исказило общую картину.

## Итоговые выводы

Наличие детей негативно влияет на вероятность просрочки кредита заемщиком.

Опыт нахождения в официальном браке, напротив , говорит о большей надежности клиента.

Люди берущие кредит для операций с недвижимостью, реже становятся должниками, по сравнению с теми, кто занимет деньги на образование или операции с автомобилем.

Задачи поставленные заказчиком были выполнены, также была предоставлена дополнительная информация.